In [37]:
import os
import re
from PIL import Image
from reportlab.pdfgen import canvas
from pathlib import Path

def extract_number_from_filename(filename):
    """ ファイル名から最後の数値部分を抽出 """
    match = re.search(r'(\d+)\.jpg$', filename)
    return int(match.group(1)) if match else 0


def natural_sort_key(s):
    """ ファイル名内の数字をキーとしてソートするためのキー関数 """
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def create_print(directory_path, output_format='pdf', margin=10, output_name='output'):
    # 画像ファイルのリストを取得
    images = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    images = sorted(images, key=extract_number_from_filename)[::-1]

    # プリントの設定（横向きA4サイズ）
    page_width, page_height = 842, 595  # A4サイズ（横向き、ピクセル単位）

    # PDFファイル生成
    c = canvas.Canvas(f"{output_name}.pdf", pagesize=(page_width, page_height))

    for page_num, page_images in enumerate(chunked(images, 10), start=1):
        for idx, img_path in enumerate(page_images):
            img = Image.open(img_path)
            
            # もし縦長の画像なら回転
            if img.height < img.width:
                img = img.rotate(90, expand=True)

            # 画像のアスペクト比を保持しながら最大サイズでリサイズ
            img_ratio = img.width / img.height
            max_width, max_height = (page_width - 6 * margin) / 5, (page_height - 3 * margin) / 2
            if img_ratio > 1:  # 横長の画像
                img_height = min(max_height, img.height)
                img_width = img_height * img_ratio
            else:  # 縦長の画像
                img_width = min(max_width, img.width)
                img_height = img_width / img_ratio

            img = img.resize((int(img_width), int(img_height)), Image.ANTIALIAS)

            # 画像をページ中央に配置            
            x = margin + (idx % 5) * (max_width + margin) + (max_width - img_width) / 2
            y = margin + (1 - idx // 5) * (max_height + margin) + (max_height - img_height) / 2

            # PDFに画像を配置
            c.drawImage(img_path, x, y, img_width, img_height)

        # ページ番号を追加
        c.drawString(page_width / 2 - 10, 15, str(page_num))
        c.showPage()

    c.save()

def chunked(iterable, size):
    """リストを指定されたサイズのチャンクに分割するヘルパー関数"""
    for i in range(0, len(iterable), size):
        yield iterable[i:i + size]

# 使用例



target_dirs = [
    "./301-318/",
    "./501-518/",
    "./701-718/",
    "./902-918/",
    "./1101-1117/",
]

for dir in target_dirs:
    create_print(dir, output_format='pdf', output_name=Path(dir).name)


# create_print('./301-318/', output_format='pdf', output_name='301-318')


/tmp/ipykernel_37126/2651716591.py:46: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((int(img_width), int(img_height)), Image.ANTIALIAS)
